In [ ]:
# Set paths

import os
import pandas as pd

main = ''



rsopen = pd.read_csv(os.path.join(main, 'rs_open_alpha_amplitude_AllBlocks.csv'))
rsclosed = pd.read_csv(os.path.join(main, 'rs_closed_alpha_amplitude_AllBlocks.csv'))

rsopen.head()
           

In [ ]:
# Load infos about EEG montage and display regions channels

from IPython.display import Image, display

# Load and display an image
image_path = os.path.join(main, "EEG_regions_electrodes.png") # Update this to the path of your image file
display(Image(filename=image_path))

channel_dict = {
        "frontal_left": [23, 24, 27, 28],
        "frontal_midline": [5, 11, 12, 16],
        "frontal_right": [3, 117, 123, 124],
        "central_left": [ 35, 36, 41, 42],
        "central_midline": [7, 31, 80, 106],
        "central_right": [93, 103, 104, 110],
        "posterior_left": [51, 52, 59, 60],
        "posterior_midline": [62, 71, 72, 76],
        "posterior_right": [85, 91, 92, 97]
                 }
 

In [4]:
#frequencies 
frequencies = ["alpha"]

In [5]:
# Generate columns of the dataframe

frequencies = ["alpha"]

columns = list(["subid","rs_block", "diagnosis", "age", "sex", "fiq"])
frequencies_parameters = []

for band in frequencies:
    parameters = [ f"{band}_amplitude"]
    frequencies_parameters.extend(parameters)
   
for region in channel_dict.keys():
    region_columns = [f"{parameter}_{region}" for parameter in frequencies_parameters]
    columns.extend(region_columns)


In [ ]:
rsclosed

In [ ]:
# RESTING STATE CLOSED
# Initialize all subjects database

data_row = [1] * len(columns)  # Example data for one row, all values set to 1",
data_dict = {col: "NA" for col, val in zip(columns, data_row)}  # LP: val is leftover I guess
all_df = pd.DataFrame([data_dict])

for rs_block in range(1, 6): 

    # subset block data
    freq_data = rsclosed[rsclosed['rs_block']==rs_block]
    freq_data= freq_data.iloc[0:].reset_index()
    freq_string = "alpha"

    # Get subj list
    sublist = set(freq_data["subid"])
    
    # Plot over subjects
    for i_sub, sub in enumerate(sublist):

        df = pd.DataFrame([data_dict])
        df.at[0, "subid"] = sub
        df.at[0, "rs_block"] = rs_block
        df.at[0, "diagnosis"] = freq_data.loc[i_sub, 'diagnosis']
        df.at[0, "age"] = freq_data.loc[i_sub, 'age']
        df.at[0, "sex"] = freq_data.loc[i_sub, 'sex']
        df.at[0, 'fiq'] = freq_data.loc[i_sub, 'fiq']                                             

        for reg in channel_dict.keys():
        
            channel_list = channel_dict[reg]
            parameters = [ f"{freq_string}_amplitude"]
        
            # Get the average for channels in the region
            for par in parameters:
                columns_to_average = [f"{par}_{ch}" for ch in channel_list]
                df.at[0, f"{par}_{reg}"] = freq_data.loc[i_sub, columns_to_average].mean()

        all_df = pd.concat([all_df, df])

# save database
closed_region_data = all_df.iloc[1:] # remove empty rom

closed_region_data.to_csv(os.path.join(main, "rs_closed_alpha_regions_AllBlocks.csv"), index=False) 
closed_region_data.head()

# Data organization-wise, you could have also had one line for every location, a "location" column, and an "amplitude" column.

In [ ]:
# RESTING STATE OPEN
# Initialize all subjects database
# LP: very important: you should avoid duplications as big as this one!
# a function would have been much better. Any bug you could spot in the previous cell would be duplicated here,
# and you would have to fix it in two places. This is a very common source of errors in programming!

data_row = [1] * len(columns)  # Example data for one row, all values set to 1",
data_dict = {col: "NA" for col, val in zip(columns, data_row)}
all_df = pd.DataFrame([data_dict])
all_df.head()


for rs_block in range(1, 6): 

    # subset block data
    freq_data = rsopen[rsopen['rs_block']==rs_block]
    freq_data= freq_data.iloc[0:].reset_index()
    freq_string = "alpha"

    # get sub list
    sublist = set(freq_data["subid"])

    for i_sub, sub in enumerate(sublist):

        df = pd.DataFrame([data_dict])
        df.at[0, "subid"] = sub
        df.at[0, "rs_block"] = rs_block
        df.at[0, "diagnosis"] = freq_data.loc[i_sub, 'diagnosis']
        df.at[0, "age"] = freq_data.loc[i_sub, 'age']
        df.at[0, "sex"] = freq_data.loc[i_sub, 'sex']
        df.at[0, 'fiq'] = freq_data.loc[i_sub, 'fiq']

        for reg in channel_dict.keys():
        
            channel_list = channel_dict[reg]
            print(channel_list)
            parameters = [ f"{freq_string}_amplitude"]
        
            # compute the average for groups of channels
            for par in parameters:
                columns_to_average = [f"{par}_{ch}" for ch in channel_list]
                df.at[0, f"{par}_{reg}"] = freq_data.loc[i_sub, columns_to_average].mean()

        all_df = pd.concat([all_df, df])


open_region_data = all_df.iloc[1:] # remove empty row
open_region_data.head()
open_region_data.to_csv(os.path.join(main, "rs_open_alpha_regions_AllBlocks.csv"), index=False) # Save database
 